In [1]:
##### from torch.utils.data import Dataset
import random
import torch
import glob as gl
import os
import random
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import InterpolationMode
import cv2
#定义数据集
class ImageDataset(Dataset):
    def __init__(self,root='',trans=None):
        super().__init__()
        self.transform = transforms.Compose(trans)
        self.A_path = os.path.join(root,"trainA/*")
        self.B_path = os.path.join(root,"trainB/*")
        self.listA = gl.glob(self.A_path)
        self.listB = gl.glob(self.B_path)
    def  __getitem__ (self, index):
        imgA_path = random.choice(self.listA)
        imgB_path = random.choice(self.listB)
        imgA = Image.open(imgA_path).convert('RGB')
        imgB = Image.open(imgB_path).convert('RGB')
        img_A = self.transform(imgA)
        img_B = self.transform(imgB)
        return img_A,img_B
    def __len__(self):
        return max(len(self.listA),len(self.listB)) 
class ImageDataset_test(Dataset):
    def __init__(self,root='',trans=None):
        super().__init__()
        self.transform = transforms.Compose(trans)
        self.A_path = os.path.join(root,"testA/*")
        self.B_path = os.path.join(root,"testB/*")
        self.listA = gl.glob(self.A_path)
        self.listB = gl.glob(self.B_path)
    def  __getitem__ (self, index):
        imgA_path = random.choice(self.listA)
        imgB_path = random.choice(self.listB)
        imgA = Image.open(imgA_path).convert('RGB')
        imgB = Image.open(imgB_path).convert('RGB')
        img_A = self.transform(imgA)
        img_B = self.transform(imgB)
        return img_A,img_B
    def __len__(self):
        return max(len(self.listA),len(self.listB))     

In [2]:
import torch
from torch import nn
from torch.nn import Parameter
#定义残差快
import torch
from torch import nn
from torchvision import transforms
from torchvision.transforms import InterpolationMode
from PIL import Image
from torch.utils.data import DataLoader
import time as t
class ResnetBlock(nn.Module):
    def __init__(self,channels,bias):
        super(ResnetBlock, self).__init__()
        self.Resblock=[]
        self.Resblock+=[nn.ReflectionPad2d(1),
                       nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=0, bias=False),
                       nn.InstanceNorm2d(channels),
                       nn.ReLU(True)]
        self.Resblock+=[nn.ReflectionPad2d(1),
                       nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=0, bias=False),
                       nn.InstanceNorm2d(channels)]
        self.conv_block=nn.Sequential(*self.Resblock)
    def forward(self,x):
        return x+self.conv_block(x)
class ResnetAdaILNBlock(nn.Module):
    def __init__(self, dim, use_bias):
        super(ResnetAdaILNBlock, self).__init__()
        self.pad1 = nn.ReflectionPad2d(1)
        self.conv1 = nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=0, bias=use_bias)
        self.norm1 = adaILN(dim)
        self.relu1 = nn.ReLU(True)
        self.pad2 = nn.ReflectionPad2d(1)
        self.conv2 = nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=0, bias=use_bias)
        self.norm2 = adaILN(dim)

    def forward(self, x, gamma, beta):
        out = self.pad1(x)
        out = self.conv1(out)
        out = self.norm1(out, gamma, beta)
        out = self.relu1(out)
        out = self.pad2(out)
        out = self.conv2(out)
        out = self.norm2(out, gamma, beta)
        return out + x
class adaILN(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super(adaILN, self).__init__()
        self.eps = eps
        self.rho = Parameter(torch.Tensor(1, num_features, 1, 1))
        self.rho.data.fill_(0.9)

    def forward(self, input, gamma, beta):
        in_mean, in_var = torch.mean(input, dim=[2, 3], keepdim=True), torch.var(input, dim=[2, 3], keepdim=True)
        out_in = (input - in_mean) / torch.sqrt(in_var + self.eps)
        ln_mean, ln_var = torch.mean(input, dim=[1, 2, 3], keepdim=True), torch.var(input, dim=[1, 2, 3], keepdim=True)
        out_ln = (input - ln_mean) / torch.sqrt(ln_var + self.eps)
        out = self.rho.expand(input.shape[0], -1, -1, -1) * out_in + (1-self.rho.expand(input.shape[0], -1, -1, -1)) * out_ln
        out = out * gamma.unsqueeze(2).unsqueeze(3) + beta.unsqueeze(2).unsqueeze(3)
        return out
class ILN(nn.Module):
    def __init__(self, num_features, eps=1e-5):
        super(ILN, self).__init__()
        self.eps = eps
        self.rho = Parameter(torch.Tensor(1, num_features, 1, 1))
        self.gamma = Parameter(torch.Tensor(1, num_features, 1, 1))
        self.beta = Parameter(torch.Tensor(1, num_features, 1, 1))
        self.rho.data.fill_(0.0)
        self.gamma.data.fill_(1.0)
        self.beta.data.fill_(0.0)

    def forward(self, input):
        in_mean, in_var = torch.mean(input, dim=[2, 3], keepdim=True), torch.var(input, dim=[2, 3], keepdim=True)
        out_in = (input - in_mean) / torch.sqrt(in_var + self.eps)
        ln_mean, ln_var = torch.mean(input, dim=[1, 2, 3], keepdim=True), torch.var(input, dim=[1, 2, 3], keepdim=True)
        out_ln = (input - ln_mean) / torch.sqrt(ln_var + self.eps)
        out = self.rho.expand(input.shape[0], -1, -1, -1) * out_in + (1-self.rho.expand(input.shape[0], -1, -1, -1)) * out_ln
        out = out * self.gamma.expand(input.shape[0], -1, -1, -1) + self.beta.expand(input.shape[0], -1, -1, -1)
        return out

#定义生成器
class Generator(nn.Module):
    def __init__(self,light=False,num_block=4):
        super(Generator,self).__init__()
        self.light=light
        self.num_block=num_block
        channels=[3,64,128,256,512]
        #编码器部分
        self.DownBlock=[]
        self.DownBlock+=[nn.ReflectionPad2d(3),nn.Conv2d(channels[0],channels[1],7,1,0, bias=False),
                  nn.InstanceNorm2d(channels[1]),nn.ReLU(True)]
        #down 下采样 2次
        for i in range(1,len(channels)-2):
            self.DownBlock+=[nn.ReflectionPad2d(1),nn.Conv2d(channels[i],channels[i+1],3,2,0, bias=False),
                      nn.InstanceNorm2d(channels[i+1]),nn.ReLU(True)]
        #定义残差块
        for j in range(self.num_block):
            self.DownBlock+=[ResnetBlock(256,bias=False)]
        #CAM
        self.gap_fc = nn.Linear(channels[3], 1, bias=False)
        self.gmp_fc = nn.Linear(channels[3], 1, bias=False)
        self.conv1x1=nn.Conv2d(channels[4],channels[3],1,1,bias=True)#512--256
        self.relu=nn.ReLU(True)
        #gamma,beta block
        self.FC = []
        if self.light:
            self.FC=[nn.Linear(channels[3],channels[3],bias=False),
                nn.ReLU(True),nn.Linear(channels[3],channels[3],bias=False),
                nn.ReLU(True)]
        else:
            self.FC=[nn.Linear(channels[1]**2*channels[3],channels[3],bias=False),
                nn.ReLU(True),
                nn.Linear(channels[3], channels[3], bias=False),
                nn.ReLU(True)]
        self.gamma = nn.Linear(channels[3],channels[3], bias=False)
        self.beta = nn.Linear(channels[3], channels[3], bias=False)
        #解码器
        # ADALIN
        self.UpBlock2 =[]
        for i in range(num_block):
            setattr(self, 'UpBlock1_' + str(i+1), ResnetAdaILNBlock(channels[3],use_bias=False))
        # up samlping
        channels.reverse()
        n=1
        for m in range(1,len(channels)-2):
            self.UpBlock2+=[nn.Upsample(scale_factor=2,mode='nearest'),#上采样的算法
                     nn.ReflectionPad2d(1),
                     nn.Conv2d(channels[m],channels[m+1],3,1,0,bias=False),
                     ILN(channels[m+1]),nn.ReLU(True)]
            n+=1
        self.UpBlock2+=[nn.ReflectionPad2d(3),nn.Conv2d(channels[n],channels[n+1], kernel_size=7, stride=1, padding=0, bias=False),
                     nn.Tanh()]
        self.DownBlock=nn.Sequential(*self.DownBlock)
        self.FC=nn.Sequential(*self.FC)
        self.UpBlock2=nn.Sequential(*self.UpBlock2)
    def forward(self,x):
        x=self.DownBlock(x)
        #输入全连接层
        gap=nn.functional.adaptive_avg_pool2d(x,1)
        gap_out=self.gap_fc(gap.view(x.shape[0],-1))
        gap_weight=list(self.gap_fc.parameters())[0]
        
        gap=x*gap_weight.unsqueeze(2).unsqueeze(3)

        gmp = nn.functional.adaptive_max_pool2d(x, 1)
        gmp_out = self.gmp_fc(gmp.view(x.shape[0], -1))
        gmp_weight = list(self.gmp_fc.parameters())[0]
        gmp = x * gmp_weight.unsqueeze(2).unsqueeze(3)
        #CAM
        cam_logit=torch.cat([gap_out,gmp_out],1)
        x=torch.cat([gap,gmp],1)
        x=self.relu(self.conv1x1(x))
        heatmap = torch.sum(x, dim=1, keepdim=True)
        if self.light:
            x_ = torch.nn.functional.adaptive_avg_pool2d(x, 1)
            x_ = self.FC(x_.view(x_.shape[0], -1))
        else:
            x_ = self.FC(x.view(x.shape[0], -1))
        gamma,beta = self.gamma(x_), self.beta(x_)
        for i in range(self.num_block):
            x = getattr(self, 'UpBlock1_' + str(i+1))(x,gamma,beta)
        out=self.UpBlock2(x)
        return out,cam_logit,heatmap
#判别器
class Discriminator(nn.Module):
    def __init__(self,n_layers=5):
        super(Discriminator,self).__init__()
        if n_layers==5:
            channels=[3,64,128,256,512,1024]
        else:
            channels=[3,64,128,256,512,1024,2048,4096]
        #编码器
        self.model=[]
        n=0
        for i in range(len(channels)-3):
            self.model+=[nn.ReflectionPad2d(1),nn.utils.spectral_norm(
            nn.Conv2d(channels[i],channels[i+1],4,2,0,bias=True)),
                     nn.LeakyReLU(0.2,True)]
            n+=1
        self.model+=[nn.ReflectionPad2d(1),nn.utils.spectral_norm(
            nn.Conv2d(channels[n],channels[n+1],4,1,0,bias=True)),
                     nn.LeakyReLU(0.2,True)]
          # Class Activation Map
        self.gap_fc=nn.utils.spectral_norm(nn.Linear(channels[n+1],1,bias=False))
        self.gmp_fc=nn.utils.spectral_norm(nn.Linear(channels[n+1],1,bias=False))
        self.conv1x1=nn.Conv2d(channels[n+2],channels[n+1],1,1,bias=True)
        self.leaky_relu=nn.LeakyReLU(0.2,True)
        #classifcation
        self.pad = nn.ReflectionPad2d(1)
        self.conv = nn.utils.spectral_norm(
            nn.Conv2d(channels[n+1], 1, kernel_size=4, stride=1, padding=0, bias=False))

        self.model = nn.Sequential(*self.model)

        # Class Activation Map

    def forward(self,x):
        x=self.model(x)
        gap = torch.nn.functional.adaptive_avg_pool2d(x, 1)
        gap_logit = self.gap_fc(gap.view(x.shape[0], -1))
        gap_weight = list(self.gap_fc.parameters())[0]
        gap = x * gap_weight.unsqueeze(2).unsqueeze(3)

        gmp = torch.nn.functional.adaptive_max_pool2d(x, 1)
        gmp_logit = self.gmp_fc(gmp.view(x.shape[0], -1))
        gmp_weight = list(self.gmp_fc.parameters())[0]
        gmp = x * gmp_weight.unsqueeze(2).unsqueeze(3)

        cam_logit = torch.cat([gap_logit, gmp_logit],1)
        x = torch.cat([gap, gmp],1)
        x = self.leaky_relu(self.conv1x1(x))
        heatmap = torch.sum(x, dim=1, keepdim=True)
        x = self.pad(x)
        x=self.conv(x)
        return x, cam_logit, heatmap

class RhoCliper(object):
        def __init__(self, min, max):
            self.clip_min = min
            self.clip_max = max
            assert min < max

        def __call__(self, module):
            if hasattr(module, 'rho'):
                w = module.rho.data
                w = w.clamp(self.clip_min, self.clip_max)
                module.rho.data = w

In [3]:
def denorm(x):
    return x * 0.5 + 0.5

def tensor2numpy(x):
    return x.detach().cpu().numpy().transpose(1,2,0)

def RGB2BGR(x):
    return cv2.cvtColor(x, cv2.COLOR_RGB2BGR)

def change_time(time):
    new_time=t.localtime(time)
    new_time=t.strftime("%Hh%Mm%Ss",new_time)
    return new_time
def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

def str2bool(x):
    return x.lower() in ('true')

def cam(x, size = 256):
    x = x - np.min(x)
    cam_img = x / np.max(x)
    cam_img = np.uint8(255 * cam_img)
    cam_img = cv2.resize(cam_img, (size, size))
    cam_img = cv2.applyColorMap(cam_img, cv2.COLORMAP_JET)
    return cam_img / 255.0
def weights_init(m):                                    
    classname = m.__class__.__name__                        
    if classname.find("Conv") != -1:                        ## find():实现查找classname中是否含有Conv字符，没有返回-1；有返回0.
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)     ## m.weight.data表示需要初始化的权重。nn.init.normal_():表示随机初始化采用正态分布，均值为0，标准差为0.02.
        if hasattr(m, "bias") and m.bias is not None:       ## hasattr():用于判断m是否包含对应的属性bias, 以及bias属性是否不为空.
            torch.nn.init.constant_(m.bias.data, 0.0)       ## nn.init.constant_():表示将偏差定义为常量0.
    elif classname.find("BatchNorm2d") != -1:               ## find():实现查找classname中是否含有BatchNorm2d字符，没有返回-1；有返回0.
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)     ## m.weight.data表示需要初始化的权重. nn.init.normal_():表示随机初始化采用正态分布，均值为0，标准差为0.02.
        torch.nn.init.constant_(m.bias.data, 0.0) 
#自定义学习率衰退
def MyLambdaLR(lr,iter):
    i=0
    lr-=(lr/(iter//2))
    return lr
        
    
    

In [4]:
import time, itertools
from torchvision import transforms
from torch.utils.data import DataLoader
from glob import glob
from PIL import Image
from torch import  nn
from torch import optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from matplotlib.pyplot import imsave
import  numpy as np
from torchvision import utils
from tqdm import tqdm
#定义UGATIT
class UGATIT(object):
    def __init__(self,args):
        self.light=args.light
        self.dataset=args.dataset
        self.mode=args.mode
         #config
        self.batch_size=args.batch_size
        self.iter = args.iter
        self.lr=args.lr
        self.decay_lr=args.decay_lr
        self.device=args.device
        self.b1=args.b1
        self.b2=args.b2
        self.path=args.path
        self.data_path=args.data_path
        self.test_path=args.test_path

         # Weight """
        self.adv_weight = args.adv_weight
        self.cycle_weight = args.cycle_weight
        self.identity_weight = args.identity_weight
        self.cam_weight = args.cam_weight
        self.weight_decay=args.weight_decay
        #img
        self.img_size=args.img_size
        self.channels=args.channels
        self.dir=args.dir
        self.print_freq=args.print_freq
        #模型
        self.G_A2B=Generator(light=self.light).to(self.device)
        self.G_B2A=Generator(light=self.light).to(self.device)
        self.D_GA=Discriminator(n_layers=7).to(self.device)
        self.D_GB=Discriminator(n_layers=7).to(self.device)
        self.D_LA=Discriminator().to(self.device)
        self.D_LB=Discriminator().to(self.device)
        #定义损失函数
        self.L1_loss = nn.L1Loss().to(self.device)
        self.MSE_loss = nn.MSELoss().to(self.device)
        self.BCE_loss = nn.BCEWithLogitsLoss().to(self.device)
        #定义优化器
        self.G_optim=optim.Adam(itertools.chain(self.G_A2B.parameters(),self.G_B2A.parameters()),lr=self.decay_lr,
                                                betas=(self.b1,self.b2),weight_decay=self.weight_decay)
        self.D_optim=optim.Adam(itertools.chain(self.D_GA.parameters(),self.D_GB.parameters(),self.D_LA.parameters(),
                                                self.D_LB.parameters()),lr=self.decay_lr,
                                        betas=(self.b1,self.b2),weight_decay=self.weight_decay)
        #定义学习率策略
#         self.G_lr_scheduler=lr_scheduler.LambdaLR(self.G_optim,lr_lambda=MyLambdaLR(self.lr,self.iter))
#         self.D_lr_scheduler=lr_scheduler.LambdaLR(self.D_optim,lr_lambda=MyLambdaLR(self.lr,self.iter))
        #rho
        self.rho=RhoCliper(0, 1)
    #读取数据集
    def load_data(self):
        train_transform =[
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Resize((self.img_size + 30, self.img_size + 30)),
            transforms.RandomCrop(self.img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ]
        test_transform =[
            transforms.Resize((self.img_size, self.img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ]

        self.train_iter=DataLoader(ImageDataset(os.path.join(self.data_path,self.dataset),trans=train_transform),batch_size=self.batch_size,
                              shuffle=False)
        self.test_iter=DataLoader(ImageDataset_test(os.path.join(self.data_path,self.dataset),trans=test_transform),batch_size=self.batch_size,
                              shuffle=False)
        # 保存模型

    def save_model(self):
        params = {}
        params["genA2B"] = self.G_A2B.state_dict()
        params["genB2A"] = self.G_B2A.state_dict()
        params["disGA"] = self.D_GA.state_dict()
        params["disGB"] = self.D_GB.state_dict()
        params["disLA"] = self.D_LA.state_dict()
        params["disLB"] = self.D_LB.state_dict()
        torch.save(params, os.path.join(self.dir, self.dataset + "model.pt"))
#     def save_lr(self):
#         lr_pamars={}
#         params['G_lr']=self.G_optim.state_dict()
#         params['D_lr']=self.D_optim.state_dict()
#         torch.save(lr_params,os.path.join(self.dir,self.dataset+'lr.pt'))
        
        #加载模型
    def load_model(self):
        params=torch.load(os.path.join(self.test_path,'selfie2animemodel.pt'))
        
        self.G_A2B.load_state_dict(params['genA2B'])
        self.G_B2A.load_state_dict(params['genB2A'])
        self.D_GA.load_state_dict(params['disGA'])
        self.D_GB.load_state_dict(params['disGB'])
        self.D_LA.load_state_dict(params['disLA'])
        self.D_LB.load_state_dict(params['disLB'])
        print("加载模型成功！")     
#         self.G_optim.load_state_dict(lr_params['G_lr'])
#         self.D_optim.load_state_dict(lr_params['D_lr'])
        print("加载学习率成功！")   
    #训练
    def train(self):
        #训练模式
        self.G_A2B.train(),self.G_B2A.train(),self.D_GA.train(),self.D_GB.train(),
        self.D_LA.train(),self.D_LB.train()
        start_time=time.time()
        for step in tqdm(range(1,10001)):
            real_A,_=next(iter(self.train_iter))
            _,real_B=next(iter(self.train_iter))
            real_A,real_B=real_A.to(self.device),real_B.to(self.device)
            #updater D
            self.D_optim.zero_grad()
            fake_B,_,_=self.G_A2B(real_A)
            fake_A,_,_=self.G_B2A(real_B)
            #forward
            real_logit_ga,real_cam_logit_ga,_=self.D_GA(real_A)
            real_logit_la,real_cam_logit_la,_=self.D_LA(real_A)
            real_logit_gb,real_cam_logit_gb,_=self.D_GB(real_B)
            real_logit_lb,real_cam_logit_lb,_=self.D_LB(real_B)


            fake_logit_ga,fake_cam_logit_ga,_=self.D_GA(fake_A)
            fake_logit_la,fake_cam_logit_la,_=self.D_LA(fake_A)
            fake_logit_gb,fake_cam_logit_gb,_=self.D_GB(fake_B)
            fake_logit_lb,fake_cam_logit_lb,_=self.D_LB(fake_B)

            #loss A
            D_loss_GA=self.MSE_loss(real_logit_ga,torch.ones_like(real_logit_ga).to(self.device))+self.MSE_loss(fake_logit_ga,torch.zeros_like(fake_logit_ga).to(self.device))
            D_loss_cam_GA=self.MSE_loss(real_cam_logit_ga,torch.ones_like(real_cam_logit_ga).to(self.device))+self.MSE_loss(fake_cam_logit_ga,torch.zeros_like(fake_cam_logit_ga).to(self.device))
            D_loss_LA=self.MSE_loss(real_logit_la,torch.ones_like(real_logit_la).to(self.device))+self.MSE_loss(fake_logit_la,torch.zeros_like(fake_logit_la)).to(self.device)
            D_loss_cam_LA=self.MSE_loss(real_cam_logit_la,torch.ones_like(real_cam_logit_la).to(self.device))+self.MSE_loss(fake_cam_logit_la,torch.zeros_like(fake_cam_logit_la).to(self.device))
            D_loss_GB=self.MSE_loss(real_logit_gb,torch.ones_like(real_logit_gb).to(self.device))+self.MSE_loss(fake_logit_gb,torch.zeros_like(fake_logit_gb).to(self.device))
            D_loss_cam_GB=self.MSE_loss(real_cam_logit_gb,torch.ones_like(real_cam_logit_gb).to(self.device))+self.MSE_loss(fake_cam_logit_gb,torch.zeros_like(fake_cam_logit_gb).to(self.device))
            D_loss_LB=self.MSE_loss(real_logit_lb,torch.ones_like(real_logit_lb).to(self.device))+self.MSE_loss(fake_logit_lb,torch.zeros_like(fake_logit_lb).to(self.device))
            D_loss_cam_LB=self.MSE_loss(real_cam_logit_lb,torch.ones_like(real_cam_logit_lb).to(self.device))+self.MSE_loss(fake_cam_logit_lb,torch.zeros_like(fake_cam_logit_lb).to(self.device))
            #D_loss
            D_loss_A=self.adv_weight*(D_loss_GA+D_loss_cam_GA+D_loss_LA+D_loss_cam_LA)
            D_loss_B=self.adv_weight*(D_loss_GB+D_loss_cam_GB+D_loss_LB+D_loss_cam_GB)
            D_loss=(D_loss_A+D_loss_B)
            D_loss.backward()
            self.D_optim.step()
            #Update G
            self.G_optim.zero_grad()

            fake_a2b,fake_cam_logit_a2b,_=self.G_A2B(real_A)
            fake_b2a,fake_cam_logit_b2a,_=self.G_B2A(real_B)
            
            fake_a2b2a,_,_=self.G_B2A(fake_a2b)
            fake_b2a2b,_,_=self.G_A2B(fake_b2a)
            
            fake_a2a, fake_cam_logit_a2a, _ = self.G_B2A(real_A)
            fake_b2b, fake_cam_logit_b2b, _ = self.G_A2B(real_B)

            fake_logit_ga,fake_cam_logit_ga,_=self.D_GA(fake_b2a)
            fake_logit_la,fake_cam_logit_la,_=self.D_LA(fake_b2a)
            fake_logit_gb,fake_cam_logit_gb,_=self.D_GB(fake_a2b)
            fake_logit_lb,fake_cam_logit_lb,_=self.D_LB(fake_a2b)
            #adv loss
            loss_adv_Ga2b=self.MSE_loss(fake_logit_gb,torch.ones_like(fake_logit_gb).to(self.device))
            loss_adv_cam_Ga2b=self.MSE_loss(fake_cam_logit_gb,torch.ones_like(fake_cam_logit_gb).to(self.device))
            loss_adv_La2b=self.MSE_loss(fake_logit_lb,torch.ones_like(fake_logit_lb).to(self.device))
            loss_adv_cam_La2b=self.MSE_loss(fake_cam_logit_lb,torch.ones_like(fake_cam_logit_lb).to(self.device))
            loss_adv_Gb2a = self.MSE_loss(fake_logit_ga, torch.ones_like(fake_logit_ga).to(self.device))
            loss_adv_cam_Gb2a = self.MSE_loss(fake_cam_logit_ga, torch.ones_like(fake_cam_logit_ga).to(self.device))
            loss_adv_Lb2a = self.MSE_loss(fake_logit_la, torch.ones_like(fake_logit_la).to(self.device))
            loss_adv_cam_Lb2a = self.MSE_loss(fake_cam_logit_la, torch.ones_like(fake_cam_logit_la).to(self.device))
            #cycle loss
            loss_cycle_b2a=self.L1_loss(fake_a2b2a,real_A).to(self.device)
            loss_cycle_a2b=self.L1_loss(fake_b2a2b,real_B).to(self.device)
            #idetity loss
            loss_identity_a2a=self.L1_loss(fake_a2a,real_A).to(self.device)
            loss_identity_b2b=self.L1_loss(fake_b2b,real_B).to(self.device)
            #CAM loss
            loss_cam_A=self.BCE_loss(fake_cam_logit_b2a,torch.ones_like(fake_cam_logit_b2a).to(self.device))+self.BCE_loss(fake_cam_logit_a2a,torch.zeros_like(fake_cam_logit_a2a).to(self.device))
            loss_cam_B=self.BCE_loss(fake_cam_logit_a2b,torch.ones_like(fake_cam_logit_a2b).to(self.device))+self.BCE_loss(fake_cam_logit_b2b,torch.zeros_like(fake_cam_logit_b2b).to(self.device))
            #total G loss
            loss_G_A=self.adv_weight*(loss_adv_Gb2a+loss_adv_cam_Gb2a+loss_adv_Lb2a+loss_adv_cam_Lb2a)+self.cycle_weight*loss_cycle_b2a+self.identity_weight*loss_identity_a2a+self.cam_weight*loss_cam_A
            loss_G_B=self.adv_weight*(loss_adv_Ga2b + loss_adv_cam_Ga2b + loss_adv_La2b + loss_adv_cam_La2b) + self.cycle_weight * loss_cycle_a2b + self.identity_weight * loss_identity_b2b + self.cam_weight * loss_cam_B
            G_loss=loss_G_A+loss_G_B
            G_loss.backward()
            self.G_optim.step()
#             if step>500000:
            self.G_optim.param_groups[0]['lr'] -= self.lr / (self.iter // 2)
            self.D_optim.param_groups[0]['lr'] -= self.lr / (self.iter // 2)
            
            #apple rho
            self.G_A2B.apply(self.rho)
            self.G_B2A.apply(self.rho)
            end_step=time.time()
            print(f"step:[{step}/{self.iter}],G_loss:{G_loss},D_loss:{D_loss},G_lr:{self.G_optim.param_groups[0]['lr']},"
                  f"D_lr:{self.D_optim.param_groups[0]['lr']},time:{change_time(end_step-start_time)}")
            if step % self.print_freq == 0:
                test_sample_num =5
                train_sample_num=5
                A2B = np.zeros((self.img_size * 7, 0, 3))
                B2A = np.zeros((self.img_size * 7, 0, 3))
                self.G_A2B.eval(), self.G_B2A.eval(), self.D_GA.eval(), self.D_GB.eval(), self.D_LA.eval(), self.D_LB.eval()
                for _ in range(train_sample_num):
                    real_A,_= next(iter(self.train_iter))
                    _,real_B= next(iter(self.train_iter))
                    real_A, real_B = real_A.to(self.device), real_B.to(self.device)
                    fake_a2b,_,fake_heatmap_a2b=self.G_A2B(real_A)
                    fake_b2a,_,fake_heatmap_b2a=self.G_B2A(real_B)

                    fake_a2b2a,_,fake_heatmap_a2b2a=self.G_B2A(fake_a2b)
                    fake_b2a2b,_,fake_heatmap_b2a2b=self.G_A2B(fake_b2a)

                    fake_a2a,_,fake_heatmap_a2a=self.G_B2A(real_A)
                    fake_b2b,_,fake_heatmap_b2b=self.G_A2B(real_B)
#                          
                    A2B=np.concatenate((A2B,np.concatenate((RGB2BGR(tensor2numpy(denorm(real_A[0]))),
                                                          cam(tensor2numpy(fake_heatmap_a2a[0]),self.img_size),
                                                          RGB2BGR(tensor2numpy(denorm(fake_a2a[0]))),
                                                          cam(tensor2numpy(fake_heatmap_a2b[0]),self.img_size),
                                                          RGB2BGR(tensor2numpy(denorm(fake_a2b[0]))),
                                                          cam(tensor2numpy(fake_heatmap_a2b2a[0]),self.img_size),
                                                          RGB2BGR(tensor2numpy(denorm(fake_a2b2a[0])))), 0)),1)
                    B2A= np.concatenate((B2A,np.concatenate((RGB2BGR(tensor2numpy(denorm(real_B[0]))),
                                                             cam(tensor2numpy(fake_heatmap_b2b[0]),
                                                                 self.img_size),
                                                             RGB2BGR(tensor2numpy(denorm(fake_b2b[0]))),
                                                             cam(tensor2numpy(fake_heatmap_b2a[0]),
                                                                 self.img_size),
                                                             RGB2BGR(tensor2numpy(denorm(fake_b2a[0]))),
                                                             cam(tensor2numpy(fake_heatmap_b2a2b[0]),
                                                                 self.img_size),RGB2BGR(tensor2numpy(denorm(fake_b2a2b[0])))), 0)), 1)
                for _ in range(test_sample_num):
                    real_A,_= next(iter(self.test_iter))
                    _,real_B= next(iter(self.test_iter))
                    real_A, real_B = real_A.to(self.device), real_B.to(self.device)
                    fake_a2b,_,fake_heatmap_a2b=self.G_A2B(real_A)
                    fake_b2a,_,fake_heatmap_b2a=self.G_B2A(real_B)

                    fake_a2b2a,_,fake_heatmap_a2b2a=self.G_B2A(fake_a2b)
                    fake_b2a2b,_,fake_heatmap_b2a2b=self.G_A2B(fake_b2a)

                    fake_a2a,_,fake_heatmap_a2a=self.G_B2A(real_A)
                    fake_b2b,_,fake_heatmap_b2b=self.G_A2B(real_B)
#                          
                    A2B=np.concatenate((A2B,np.concatenate((RGB2BGR(tensor2numpy(denorm(real_A[0]))),
                                                          cam(tensor2numpy(fake_heatmap_a2a[0]),self.img_size),
                                                          RGB2BGR(tensor2numpy(denorm(fake_a2a[0]))),
                                                          cam(tensor2numpy(fake_heatmap_a2b[0]),self.img_size),
                                                          RGB2BGR(tensor2numpy(denorm(fake_a2b[0]))),
                                                          cam(tensor2numpy(fake_heatmap_a2b2a[0]),self.img_size),
                                                          RGB2BGR(tensor2numpy(denorm(fake_a2b2a[0])))), 0)),1)
                    B2A= np.concatenate((B2A,np.concatenate((RGB2BGR(tensor2numpy(denorm(real_B[0]))),
                                                             cam(tensor2numpy(fake_heatmap_b2b[0]),self.img_size),
                                                             RGB2BGR(tensor2numpy(denorm(fake_b2b[0]))),
                                                             cam(tensor2numpy(fake_heatmap_b2a[0]),self.img_size),
                                                             RGB2BGR(tensor2numpy(denorm(fake_b2a[0]))),
                                                             cam(tensor2numpy(fake_heatmap_b2a2b[0]),self.img_size),
                                                             RGB2BGR(tensor2numpy(denorm(fake_b2a2b[0])))), 0)), 1)
                    #保存训练测试图片
                cv2.imwrite(os.path.join(self.dir, self.dataset, 'img', 'A2B_%07d.png' % step), A2B * 255.0)
                cv2.imwrite(os.path.join(self.dir, self.dataset, 'img', 'B2A_%07d.png' % step), B2A * 255.0)
                self.G_A2B.train(), self.G_B2A.train(), self.D_GA.train(), self.D_GB.train(), self.D_LA.train(), self.D_LB.train()
            if step%self.print_freq==0:
                self.save_model()
    
                print("保存模型和学习率成功！")
    #测试
    def test(self):
        print("读取模型！")
        self.load_model()
        img=[]
        self.G_A2B.eval(),self.G_B2A.eval()
        for j in range(1):
            for i,(x,y) in tqdm(enumerate(self.test_iter)):
                x,y=x.to(self.device),y.to(self.device)
                fake_A2B=denorm(self.G_A2B(x)[0][0])
                fake_B2A=denorm(self.G_B2A(y)[0][0])
                fake_A2B=RGB2BGR(tensor2numpy(fake_A2B))
                fake_B2A=RGB2BGR(tensor2numpy(fake_B2A))
                cv2.imwrite(f"{os.path.join(self.dir, self.dataset, 'test','testA2B' ,'A2B')}.{str(i).zfill(j)}.jpg", fake_A2B * 255.0)
                cv2.imwrite(f"{os.path.join(self.dir, self.dataset, 'test','testB2A' ,'B2A')}.{str(i).zfill(j)}.jpg", fake_B2A * 255.0)
                
#                 utils.make_grid(x, nrow=16, normalize=True)
#                 utils.make_grid(y, nrow=16, normalize=True)
#                 utils.make_grid(fake_A2B,nrow=16,normalize=True)
#                 utils.make_grid(fake_B2A,nrow=16,normalize=True)
#     #                 fake_A2B=torch.cat((x,fake_A2B),0)
#     #                 fake_B2A=torch.cat((y,fake_B2A),0)
#                 utils.save_image(fake_A2B,f"{os.path.join(self.dir,self.dataset,'test','testA2B')}A2B_{str(i).zfill(j)}.jpg")
#                 utils.save_image(fake_B2A, f"{os.path.join(self.dir, self.dataset,'test','testB2A')}B2A_{str(i).zfill(j)}.jpg")


In [5]:
import argparse
import torch, gc
import os
#配置
def parse_args():
    desc = "Pytorch implementation of U-GAT-IT"
    parser = argparse.ArgumentParser(description=desc)
    parser.add_argument('--mode',type=str,default='train')
    parser.add_argument('--light',type=bool,default=True)
    parser.add_argument('--dataset',type=str,default='selfie2anime')
    parser.add_argument('--data_path',type=str,default='../input/')
    parser.add_argument('--batch_size',type=int,default=1)
    parser.add_argument('--iter',type=int,default=1000000)
    parser.add_argument('--lr',type=float,default=0.0001)
    parser.add_argument('--decay_lr',type=float,default=7.999999999798275e-06)#6
    parser.add_argument('--b1',type=float,default=0.5)
    parser.add_argument('--b2',type=float,default=0.999)
    parser.add_argument('--device',type=str,default='cuda',choices=['cpu','cuda'])
    parser.add_argument('--img_size',type=int,default=256)
    parser.add_argument('--channels',type=int,default=3)
    parser.add_argument('--dir',type=str,default='results')
    parser.add_argument('--print_freq',type=int,default='10000')
    parser.add_argument('--weight_decay', type=float, default=0.0001, help='The weight decay')
    parser.add_argument('--adv_weight', type=int, default=1, help='Weight for GAN')
    parser.add_argument('--cycle_weight', type=int, default=10, help='Weight for Cycle')
    parser.add_argument('--identity_weight', type=int, default=10, help='Weight for Identity')
    parser.add_argument('--cam_weight', type=int, default=1000, help='Weight for CAM')
    parser.add_argument('--path',type=str,default='dataset')
    parser.add_argument('--test_path',type=str,default='../input/model100w')
    parser.add_argument('--retrain',type=bool,default=True)
    parser.add_argument('--mutil_gpu',type=bool,default=False)
    return check_args(parser.parse_args(args=[]))
def check_args(args):
    check_folder(os.path.join(args.dir, args.dataset, 'model'))
    check_folder(os.path.join(args.dir, args.dataset, 'img'))
    check_folder(os.path.join(args.dir, args.dataset, 'test'))
    check_folder(os.path.join(args.dir, args.dataset, 'test','testA2B'))
    check_folder(os.path.join(args.dir, args.dataset, 'test','testB2A'))
    return args
def main():
   gc.collect()
   torch.cuda.empty_cache()
   args=parse_args()
   if args.mutil_gpu:
        os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"
        device_ids = [0, 1]
   if args is None:
       exit
   gan=UGATIT(args)
   gan.load_data()
   if args.mode=='train':
       if args.retrain:
            gan.load_model()
       print(f"training on {args.device}")
       gan.train()
       print("train haved finished")
   if args.mode=='test':
       gan.test()
       print("test haved finished")
if __name__=="__main__":
    main()

读取模型！
加载模型成功！
加载学习率成功！


60it [00:08,  6.71it/s]

test haved finished


In [6]:
!nvidia-smi

Wed Nov 23 10:39:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    37W / 250W |   2981MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
import os
import zipfile
def file2zip(packagePath, zipPath):
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()
if __name__ == "__main__":
    # 文件夹路径
    packagePath = './'
    zipPath = './testab.zip'
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")

打包完成


In [8]:

# !pip install nicefid==0.1.0 -i https://pypi.tuna.tsinghua.edu.cn/simple/

# import nicefid

# features_generated = nicefid.Features.from_iterator('../input/selfie2anime/trainB')
# features_real = nicefid.Features.from_directory('../input/selfie2anime/trainA')

# fid = nicefid.compute_fid(features_generated, features_real)
# kid = nicefid.compute_kid(features_generated, features_real)